In [1]:
import datajoint as dj
dj.config['database.host'] = 'localhost'
dj.config["stores"]= {
        "lfp_store": {
            "protocol": "file",
            "stage": "data/lfp_dj_data_temp",
            "location": "data/lfp_dj_data_temp"
        }
}

In [2]:
from pipeline import experiment
from pynwb import NWBHDF5IO
from franklab_nwb_extensions import fl_extension
import numpy as np
%reload_ext autoreload
%autoreload 2

Please enter DataJoint username: shan
Please enter DataJoint password: ········
Connecting shan@localhost:3306


In [41]:
io = NWBHDF5IO('../data/original_data/bon_04.nwb', 'r', load_namespaces=True)
nwbfile = io.read()
               
nwbfile
io.close()

/Users/shanshen/anaconda3/envs/nwb_datajoint/lib/python3.7/site-packages/hdmf/spec/namespace.py:456: UserWarning: ignoring namespace 'hdmf-common' because it already exists
  warn("ignoring namespace '%s' because it already exists" % ns['name'])
/Users/shanshen/anaconda3/envs/nwb_datajoint/lib/python3.7/site-packages/hdmf/spec/namespace.py:456: UserWarning: ignoring namespace 'core' because it already exists
  warn("ignoring namespace '%s' because it already exists" % ns['name'])
/Users/shanshen/anaconda3/envs/nwb_datajoint/lib/python3.7/site-packages/hdmf/spec/namespace.py:456: UserWarning: ignoring namespace 'franklab' because it already exists
  warn("ignoring namespace '%s' because it already exists" % ns['name'])


In [4]:
experiment.Lab.insert1((nwbfile.lab, nwbfile.institution), skip_duplicates=True)

In [5]:
experiment.Experimenter.insert1((nwbfile.experimenter), skip_duplicates=True)

In [6]:
subj = nwbfile.subject.fields

In [26]:
subj['subject_description'] = subj.pop('description')

In [27]:
subj

{'genotype': 'WT',
 'sex': 'M',
 'species': 'rat',
 'subject_id': 'Bond',
 'weight': 'unknown',
 'subject_description': 'Long Evans Rat'}

In [29]:
subj.pop('weight')

'unknown'

In [30]:
experiment.Subject.insert1(subj)

In [34]:
nwbfile.experiment_description

'Tetrode recordings from rat during rest and task performance'

In [35]:
experiment.Session.insert1(
(nwbfile.subject.subject_id, nwbfile.session_id, nwbfile.experimenter, nwbfile.experiment_description)
)

In [10]:
experiment.ProbeInsertion()

subject_id,session_id,insertion_number
Bond,Bon0004,1
Bond,Bon0004,2
Bond,Bon0004,3
Bond,Bon0004,4
Bond,Bon0004,5
Bond,Bon0004,6
Bond,Bon0004,7
Bond,Bon0004,8
Bond,Bon0004,10
Bond,Bon0004,11


In [4]:
probe_insertions = [
    dict(subject_id=nwbfile.subject.subject_id, 
         session_id=nwbfile.session_id,
         insertion_number=int(series['group_name'])) 
    for id, series in nwbfile.ec_electrodes.to_dataframe().iterrows()]

experiment.ProbeInsertion.insert(probe_insertions, skip_duplicates=True)

In [5]:
lfp_dict = nwbfile.acquisition['LFP']['electrical_series'].fields
lfp_dict

{'resolution': -1.0,
 'comments': 'no comments',
 'description': 'no description',
 'conversion': 1e-06,
 'unit': 'volts',
 'data': <HDF5 dataset "data": shape (27, 1416029), type "<i2">,
 'timestamps': <HDF5 dataset "timestamps": shape (1416029,), type "<f8">,
 'timestamps_unit': 'seconds',
 'interval': 1,
 'electrodes': <hdmf.common.table.DynamicTableRegion at 0xa2379c910>}

In [19]:
np.array(lfp_dict['data'])

array([[ 237,  204,  151, ..., -141, -128, -119],
       [ 258,  240,  196, ..., -147, -132, -110],
       [-129, -129, -142, ..., -106, -103, -100],
       ...,
       [-145, -134, -125, ...,   50,   60,   51],
       [-102, -109, -106, ..., -115,  -92,  -69],
       [ -16,   -7,  -31, ...,   25,   33,    2]], dtype=int16)

In [6]:
ignored_fields = ['electrodes', 'data']

In [7]:
lfp_entry = {f'lfp_{key_name}': value for key_name, value in lfp_dict.items() if key_name not in ignored_fields}

In [8]:
lfp_entry['lfp_timestamps'] = np.array(lfp_entry['lfp_timestamps'])

In [15]:
experiment.LFP()

subject_id,session_id,insertion_number,lfp_timestamps,lfp_timestamps_unit,lfp_unit,lfp_resolution,lfp_conversion,lfp_interval,lfp_description,lfp_comments
Bond,Bon0004,1,=BLOB=,seconds,volts,-1.0,1e-06,1,no description,no comments
Bond,Bon0004,2,=BLOB=,seconds,volts,-1.0,1e-06,1,no description,no comments
Bond,Bon0004,3,=BLOB=,seconds,volts,-1.0,1e-06,1,no description,no comments
Bond,Bon0004,4,=BLOB=,seconds,volts,-1.0,1e-06,1,no description,no comments
Bond,Bon0004,5,=BLOB=,seconds,volts,-1.0,1e-06,1,no description,no comments
Bond,Bon0004,6,=BLOB=,seconds,volts,-1.0,1e-06,1,no description,no comments
Bond,Bon0004,7,=BLOB=,seconds,volts,-1.0,1e-06,1,no description,no comments
Bond,Bon0004,8,=BLOB=,seconds,volts,-1.0,1e-06,1,no description,no comments
Bond,Bon0004,10,=BLOB=,seconds,volts,-1.0,1e-06,1,no description,no comments
Bond,Bon0004,11,=BLOB=,seconds,volts,-1.0,1e-06,1,no description,no comments


In [14]:
for probe_insertion in probe_insertions:
    experiment.LFP.insert1(dict(**lfp_entry, **probe_insertion), 
                           allow_direct_insert=True, skip_duplicates=True)

In [16]:
time_series = (experiment.LFP & 'insertion_number=2').fetch1('lfp_timestamps')

In [17]:
time_series

array([2460.9967    , 2460.99736667, 2460.99803333, ..., 3405.01403353,
       3405.01470019, 3405.01536686])

In [35]:
[dict(electrode_id=int(row.index.tolist()[0]), group_name=int(row['group_name'].values[0])) 
 for row in nwbfile.acquisition['LFP']['electrical_series'].fields['electrodes']]

[{'electrode_id': 0, 'group_name': 1},
 {'electrode_id': 4, 'group_name': 2},
 {'electrode_id': 8, 'group_name': 3},
 {'electrode_id': 12, 'group_name': 4},
 {'electrode_id': 16, 'group_name': 5},
 {'electrode_id': 20, 'group_name': 6},
 {'electrode_id': 24, 'group_name': 7},
 {'electrode_id': 28, 'group_name': 8},
 {'electrode_id': 32, 'group_name': 10},
 {'electrode_id': 36, 'group_name': 11},
 {'electrode_id': 40, 'group_name': 12},
 {'electrode_id': 44, 'group_name': 13},
 {'electrode_id': 48, 'group_name': 14},
 {'electrode_id': 52, 'group_name': 15},
 {'electrode_id': 56, 'group_name': 17},
 {'electrode_id': 60, 'group_name': 18},
 {'electrode_id': 64, 'group_name': 19},
 {'electrode_id': 68, 'group_name': 20},
 {'electrode_id': 72, 'group_name': 21},
 {'electrode_id': 76, 'group_name': 22},
 {'electrode_id': 80, 'group_name': 23},
 {'electrode_id': 84, 'group_name': 24},
 {'electrode_id': 88, 'group_name': 25},
 {'electrode_id': 92, 'group_name': 27},
 {'electrode_id': 96, 'grou

In [37]:
lfp_data = np.array(lfp_dict['data'])
for idx, row in enumerate(nwbfile.acquisition['LFP']['electrical_series'].fields['electrodes']):
    lfp = (experiment.LFP & f'insertion_number={int(row["group_name"].values[0])}').fetch1('KEY')
    lfp_channel_data = lfp_data[idx]
    
    lfp_channel=dict(**lfp, 
                     electrode_id=int(row.index.tolist()[0]),
                     lfp=lfp_channel_data)
    experiment.LFP.Channel.insert1(lfp_channel, allow_direct_insert=True)

In [38]:
experiment.LFP.Channel()

subject_id,session_id,insertion_number,electrode_id should be -> ElectrodeConfig.Electrode,lfp recorded lfp at this electrode
Bond,Bon0004,1,0,=BLOB=
Bond,Bon0004,2,4,=BLOB=
Bond,Bon0004,3,8,=BLOB=
Bond,Bon0004,4,12,=BLOB=
Bond,Bon0004,5,16,=BLOB=
Bond,Bon0004,6,20,=BLOB=
Bond,Bon0004,7,24,=BLOB=
Bond,Bon0004,8,28,=BLOB=
Bond,Bon0004,10,32,=BLOB=
Bond,Bon0004,11,36,=BLOB=


In [39]:
lfp = (experiment.LFP.Channel & 'insertion_number=2').fetch1('lfp')

In [40]:
lfp

array([ 258,  240,  196, ..., -147, -132, -110], dtype=int16)